In [3]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score

### FLDA for 2-D data

In [4]:
p = 0.4
D = 2
means = [np.repeat(0, D).reshape(-1), np.repeat(1, D).reshape(-1)]
covariance = np.identity(D)
priors = [p, 1-p]
train_size, test_size = 2000, 1000

X_train_0 = np.random.multivariate_normal(mean=means[0], cov=covariance, size=int(train_size*priors[0]))
X_train_1 = np.random.multivariate_normal(mean=means[1], cov=covariance, size=int(train_size*priors[1]))
X_test_0 = np.random.multivariate_normal(mean=means[0], cov=covariance, size=int(test_size*priors[0]))
X_test_1 = np.random.multivariate_normal(mean=means[1], cov=covariance, size=int(test_size*priors[1]))
X_train = np.vstack((X_train_0, X_train_1))
X_test = np.vstack((X_test_0, X_test_1))

y_train = np.vstack((
    np.repeat(0, int(train_size*priors[0])).reshape(-1, 1),
    np.repeat(1, int(train_size*priors[1])).reshape(-1, 1)
))
y_test = np.vstack((
    np.repeat(0, int(test_size*priors[0])).reshape(-1, 1),
    np.repeat(1, int(test_size*priors[1])).reshape(-1, 1)
))

M_0 = X_train_0.mean(axis=0)
M_1 = X_train_1.mean(axis=0)

cov_0 = np.zeros((D, D))
for i in range(X_train_0.shape[0]):
    W = (X_train_0[i] - M_0).reshape(-1, 1)
    cov_0 += np.matmul(W, W.T)

cov_1 = np.zeros((D, D))
for i in range(X_train_1.shape[0]):
    W = (X_train_1[i] - M_1).reshape(-1, 1)
    cov_1 += np.matmul(W, W.T)

S_w = cov_0 + cov_1
S_w_inv = np.linalg.pinv(S_w)
W = np.matmul(S_w_inv, (M_1 - M_0)).reshape(-1, 1)
W = W/np.linalg.norm(W,2)

def predict(x, W, b):
    return np.matmul(W.T, x) + b


# Line Search
best_b, best_acc = 0, 0
lower = np.matmul(W.T, M_0)[0] - 5
upper = np.matmul(W.T, M_1)[0] + 5
for b in np.linspace(lower, upper, 1000):
    acc = 0
    for (i, X_i) in enumerate(X_train_0):
        pred = predict(X_i.reshape(-1, 1), W, b)
        if pred < 0:
            acc += 1

    for (i, X_i) in enumerate(X_train_1):
        pred = predict(X_i.reshape(-1, 1), W, b)
        if pred >= 0:
            acc += 1

    if acc > best_acc:
        best_acc = acc
        best_b = b
    
    # print(f'Bias {b}, Acc: {acc/X_train.shape[0]}, Best: {best_acc/X_train.shape[0]}')

print('Train acc. =', best_acc/X_train.shape[0])

b = best_b
acc = 0
for (i, X_i) in enumerate(X_test_0):
    pred = predict(X_i.reshape(-1, 1), W, b)
    if pred < 0:
        acc += 1

for (i, X_i) in enumerate(X_test_1):
    pred = predict(X_i.reshape(-1, 1), W, b)
    if pred >= 0:
        acc += 1

print(f'Test acc. = {acc/X_test.shape[0]}')

# f1-score
y_pred = []
for X_i in X_train:
    pred = predict(X_i.reshape(-1, 1), W, b)
    if pred < 0:
        y_pred.append(0)
    else:
        y_pred.append(1)
y_pred = np.array(y_pred)
print('F1-Score (train) = ', f1_score(y_train, y_pred))

y_pred = []
for X_i in X_test:
    pred = predict(X_i.reshape(-1, 1), W, b)
    if pred < 0:
        y_pred.append(0)
    else:
        y_pred.append(1)
y_pred = np.array(y_pred)

print('F1-Score (test) = ', f1_score(y_test, y_pred))
confusion_matrix(y_test, y_pred)

Train acc. = 0.787
Test acc. = 0.775
F1-Score (train) =  0.8262642740619903
F1-Score (test) =  0.815422477440525


array([[278, 122],
       [103, 497]])

In [5]:
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd

x = np.linspace(-3, 3, 100)
y = -(W[0][0]*x + b)/(W[1][0])

fig = px.scatter(x=X_train_0[:, 0], y=X_train_0[:, 1])
fig.add_scatter(x=X_train_1[:, 0], y=X_train_1[:, 1], mode='markers', name='class_1')
fig.add_scatter(x=x, y=y, name='decision boundary')

### Same Mean, Different Covariances

In [12]:
p = 0.5
D = 2
means = [np.repeat(0, D).reshape(-1), np.repeat(0, D).reshape(-1)]
covariance = np.identity(D)
covariance_1 = np.array([
    [1, 0.9], [0.9, 1]
])
priors = [p, 1-p]
train_size, test_size = 2000, 1000

X_train_0 = np.random.multivariate_normal(mean=means[0], cov=covariance, size=int(train_size*priors[0]))
X_train_1 = np.random.multivariate_normal(mean=means[1], cov=covariance_1, size=int(train_size*priors[1]))
X_test_0 = np.random.multivariate_normal(mean=means[0], cov=covariance, size=int(test_size*priors[0]))
X_test_1 = np.random.multivariate_normal(mean=means[1], cov=covariance_1, size=int(test_size*priors[1]))
X_train = np.vstack((X_train_0, X_train_1))
X_test = np.vstack((X_test_0, X_test_1))

y_train = np.vstack((
    np.repeat(0, int(train_size*priors[0])).reshape(-1, 1),
    np.repeat(1, int(train_size*priors[1])).reshape(-1, 1)
))
y_test = np.vstack((
    np.repeat(0, int(test_size*priors[0])).reshape(-1, 1),
    np.repeat(1, int(test_size*priors[1])).reshape(-1, 1)
))

M_0 = X_train_0.mean(axis=0)
M_1 = X_train_1.mean(axis=0)

cov_0 = np.zeros((D, D))
for i in range(X_train_0.shape[0]):
    W = (X_train_0[i] - M_0).reshape(-1, 1)
    cov_0 += np.matmul(W, W.T)

cov_1 = np.zeros((D, D))
for i in range(X_train_1.shape[0]):
    W = (X_train_1[i] - M_1).reshape(-1, 1)
    cov_1 += np.matmul(W, W.T)

S_w = cov_0 + cov_1
S_w_inv = np.linalg.pinv(S_w)
W = np.matmul(S_w_inv, (M_1 - M_0)).reshape(-1, 1)
W = W/np.linalg.norm(W,2)

def predict(x, W, b):
    return np.matmul(W.T, x) + b


# Line Search
best_b, best_acc = 0, 0
lower = np.matmul(W.T, M_0)[0] - 5
upper = np.matmul(W.T, M_1)[0] + 5
for b in np.linspace(lower, upper, 1000):
    acc = 0
    for (i, X_i) in enumerate(X_train_0):
        pred = predict(X_i.reshape(-1, 1), W, b)
        if pred < 0:
            acc += 1

    for (i, X_i) in enumerate(X_train_1):
        pred = predict(X_i.reshape(-1, 1), W, b)
        if pred >= 0:
            acc += 1

    if acc > best_acc:
        best_acc = acc
        best_b = b
    
    # print(f'Bias {b}, Acc: {acc/X_train.shape[0]}, Best: {best_acc/X_train.shape[0]}')

print('Train acc. =', best_acc/X_train.shape[0])

b = best_b
acc = 0
for (i, X_i) in enumerate(X_test_0):
    pred = predict(X_i.reshape(-1, 1), W, b)
    if pred < 0:
        acc += 1

for (i, X_i) in enumerate(X_test_1):
    pred = predict(X_i.reshape(-1, 1), W, b)
    if pred >= 0:
        acc += 1

print(f'Test acc. = {acc/X_test.shape[0]}')

# f1-score
y_pred = []
for X_i in X_train:
    pred = predict(X_i.reshape(-1, 1), W, b)
    if pred < 0:
        y_pred.append(0)
    else:
        y_pred.append(1)
y_pred = np.array(y_pred)
print('F1-Score (train) = ', f1_score(y_train, y_pred))

y_pred = []
for X_i in X_test:
    pred = predict(X_i.reshape(-1, 1), W, b)
    if pred < 0:
        y_pred.append(0)
    else:
        y_pred.append(1)
y_pred = np.array(y_pred)

print('F1-Score (test) = ', f1_score(y_test, y_pred))
confusion_matrix(y_test, y_pred)

Train acc. = 0.6395
Test acc. = 0.613
F1-Score (train) =  0.7259597111364501
F1-Score (test) =  0.706150341685649


array([[148, 352],
       [ 35, 465]])

In [13]:
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd

x = np.linspace(-3, 3, 100)
y = -(W[0][0]*x + b)/(W[1][0])

fig = px.scatter(x=X_train_0[:, 0], y=X_train_0[:, 1])
fig.add_scatter(x=X_train_1[:, 0], y=X_train_1[:, 1], mode='markers', name='class_1')
fig.add_scatter(x=x, y=y, name='decision boundary')

#### Transforming Data

In [8]:
X_train_0 = np.random.multivariate_normal(mean=means[0], cov=covariance, size=int(train_size*priors[0]))
X_train_0 = np.append(X_train_0, (X_train_0[:, 0]**2).reshape(-1, 1), axis=1)
X_train_0 = np.append(X_train_0, (X_train_0[:, 1]**2).reshape(-1, 1), axis=1)
X_train_0 = np.append(X_train_0, np.multiply(X_train_0[:, 0], X_train_0[:, 1]).reshape(-1, 1), axis=1)
X_train_0.shape, X_train_0[0]

((1000, 5),
 array([ 2.35515293, -1.44066232,  5.54674533,  2.07550791, -3.39298008]))

In [9]:
p = 0.5
D = 2
means = [np.repeat(0, D).reshape(-1), np.repeat(0, D).reshape(-1)]
covariance = np.identity(D)
covariance_1 = np.array([
    [1, 0.9], [0.9, 1]
])
priors = [p, 1-p]
train_size, test_size = 2000, 1000

D = 5 

X_train_0 = np.random.multivariate_normal(mean=means[0], cov=covariance, size=int(train_size*priors[0]))
X_train_0 = np.random.multivariate_normal(mean=means[0], cov=covariance, size=int(train_size*priors[0]))
X_train_0 = np.append(X_train_0, (X_train_0[:, 0]**2).reshape(-1, 1), axis=1)
X_train_0 = np.append(X_train_0, (X_train_0[:, 1]**2).reshape(-1, 1), axis=1)
X_train_0 = np.append(X_train_0, np.multiply(X_train_0[:, 0], X_train_0[:, 1]).reshape(-1, 1), axis=1)

X_train_1 = np.random.multivariate_normal(mean=means[1], cov=covariance_1, size=int(train_size*priors[1]))
X_train_1 = np.random.multivariate_normal(mean=means[1], cov=covariance_1, size=int(train_size*priors[1]))
X_train_1 = np.append(X_train_1, (X_train_1[:, 0]**2).reshape(-1, 1), axis=1)
X_train_1 = np.append(X_train_1, (X_train_1[:, 1]**2).reshape(-1, 1), axis=1)
X_train_1 = np.append(X_train_1, np.multiply(X_train_1[:, 0], X_train_1[:, 1]).reshape(-1, 1), axis=1)

X_test_0 = np.random.multivariate_normal(mean=means[0], cov=covariance, size=int(test_size*priors[0]))
X_test_0 = np.random.multivariate_normal(mean=means[0], cov=covariance, size=int(test_size*priors[0]))
X_test_0 = np.append(X_test_0, (X_test_0[:, 0]**2).reshape(-1, 1), axis=1)
X_test_0 = np.append(X_test_0, (X_test_0[:, 1]**2).reshape(-1, 1), axis=1)
X_test_0 = np.append(X_test_0, np.multiply(X_test_0[:, 0], X_test_0[:, 1]).reshape(-1, 1), axis=1)

X_test_1 = np.random.multivariate_normal(mean=means[1], cov=covariance_1, size=int(test_size*priors[1]))
X_test_1 = np.random.multivariate_normal(mean=means[1], cov=covariance_1, size=int(test_size*priors[1]))
X_test_1 = np.append(X_test_1, (X_test_1[:, 0]**2).reshape(-1, 1), axis=1)
X_test_1 = np.append(X_test_1, (X_test_1[:, 1]**2).reshape(-1, 1), axis=1)
X_test_1 = np.append(X_test_1, np.multiply(X_test_1[:, 0], X_test_1[:, 1]).reshape(-1, 1), axis=1)

X_train = np.vstack((X_train_0, X_train_1))
X_test = np.vstack((X_test_0, X_test_1))

y_train = np.vstack((
    np.repeat(0, int(train_size*priors[0])).reshape(-1, 1),
    np.repeat(1, int(train_size*priors[1])).reshape(-1, 1)
))
y_test = np.vstack((
    np.repeat(0, int(test_size*priors[0])).reshape(-1, 1),
    np.repeat(1, int(test_size*priors[1])).reshape(-1, 1)
))

M_0 = X_train_0.mean(axis=0)
M_1 = X_train_1.mean(axis=0)

cov_0 = np.zeros((D, D))
for i in range(X_train_0.shape[0]):
    W = (X_train_0[i] - M_0).reshape(-1, 1)
    cov_0 += np.matmul(W, W.T)

cov_1 = np.zeros((D, D))
for i in range(X_train_1.shape[0]):
    W = (X_train_1[i] - M_1).reshape(-1, 1)
    cov_1 += np.matmul(W, W.T)

S_w = cov_0 + cov_1
S_w_inv = np.linalg.pinv(S_w)
W = np.matmul(S_w_inv, (M_1 - M_0)).reshape(-1, 1)
W = W/np.linalg.norm(W,2)

def predict(x, W, b):
    return np.matmul(W.T, x) + b


# Line Search
best_b, best_acc = 0, 0
lower = np.matmul(W.T, M_0)[0] - 5
upper = np.matmul(W.T, M_1)[0] + 5
for b in np.linspace(lower, upper, 1000):
    acc = 0
    for (i, X_i) in enumerate(X_train_0):
        pred = predict(X_i.reshape(-1, 1), W, b)
        if pred < 0:
            acc += 1

    for (i, X_i) in enumerate(X_train_1):
        pred = predict(X_i.reshape(-1, 1), W, b)
        if pred >= 0:
            acc += 1

    if acc > best_acc:
        best_acc = acc
        best_b = b
    
    # print(f'Bias {b}, Acc: {acc/X_train.shape[0]}, Best: {best_acc/X_train.shape[0]}')

print('Train acc. =', best_acc/X_train.shape[0])

b = best_b
acc = 0
for (i, X_i) in enumerate(X_test_0):
    pred = predict(X_i.reshape(-1, 1), W, b)
    if pred < 0:
        acc += 1

for (i, X_i) in enumerate(X_test_1):
    pred = predict(X_i.reshape(-1, 1), W, b)
    if pred >= 0:
        acc += 1

print(f'Test acc. = {acc/X_test.shape[0]}')

# f1-score
y_pred = []
for X_i in X_train:
    pred = predict(X_i.reshape(-1, 1), W, b)
    if pred < 0:
        y_pred.append(0)
    else:
        y_pred.append(1)
y_pred = np.array(y_pred)
print('F1-Score (train) = ', f1_score(y_train, y_pred))

y_pred = []
for X_i in X_test:
    pred = predict(X_i.reshape(-1, 1), W, b)
    if pred < 0:
        y_pred.append(0)
    else:
        y_pred.append(1)
y_pred = np.array(y_pred)

print('F1-Score (test) = ', f1_score(y_test, y_pred))
confusion_matrix(y_test, y_pred)

Train acc. = 0.7395
Test acc. = 0.744
F1-Score (train) =  0.7793307920372724
F1-Score (test) =  0.7785467128027682


array([[294, 206],
       [ 50, 450]])

In [10]:
x = np.linspace(-3, 3, 100)
y = -(W[0][0]*x + b)/(W[1][0])

fig = px.scatter(x=X_train_0[:, 0], y=X_train_0[:, 1])
fig.add_scatter(x=X_train_1[:, 0], y=X_train_1[:, 1], mode='markers', name='class_1')
fig.add_scatter(x=x, y=y, name='decision boundary')

### Different Means, Different Covariances

In [14]:
p = 0.5
D = 2
means = [np.array([3, 6]), np.array([3, -2])]
covariance = np.array([
    [0.5, 0], [0, 2]
])
covariance_1 = np.array([
    [2, 0], [0, 2]
])
priors = [p, 1-p]
train_size, test_size = 4000, 2000

D = 2

X_train_0 = np.random.multivariate_normal(mean=means[0], cov=covariance, size=int(train_size*priors[0]))
# X_train_0 = np.random.multivariate_normal(mean=means[0], cov=covariance, size=int(train_size*priors[0]))
# X_train_0 = np.append(X_train_0, (X_train_0[:, 0]**2).reshape(-1, 1), axis=1)
# X_train_0 = np.append(X_train_0, (X_train_0[:, 1]**2).reshape(-1, 1), axis=1)
# X_train_0 = np.append(X_train_0, np.multiply(X_train_0[:, 0], X_train_0[:, 1]).reshape(-1, 1), axis=1)

X_train_1 = np.random.multivariate_normal(mean=means[1], cov=covariance_1, size=int(train_size*priors[1]))
# X_train_1 = np.random.multivariate_normal(mean=means[1], cov=covariance_1, size=int(train_size*priors[1]))
# X_train_1 = np.append(X_train_1, (X_train_1[:, 0]**2).reshape(-1, 1), axis=1)
# X_train_1 = np.append(X_train_1, (X_train_1[:, 1]**2).reshape(-1, 1), axis=1)
# X_train_1 = np.append(X_train_1, np.multiply(X_train_1[:, 0], X_train_1[:, 1]).reshape(-1, 1), axis=1)

X_test_0 = np.random.multivariate_normal(mean=means[0], cov=covariance, size=int(test_size*priors[0]))
# X_test_0 = np.random.multivariate_normal(mean=means[0], cov=covariance, size=int(test_size*priors[0]))
# X_test_0 = np.append(X_test_0, (X_test_0[:, 0]**2).reshape(-1, 1), axis=1)
# X_test_0 = np.append(X_test_0, (X_test_0[:, 1]**2).reshape(-1, 1), axis=1)
# X_test_0 = np.append(X_test_0, np.multiply(X_test_0[:, 0], X_test_0[:, 1]).reshape(-1, 1), axis=1)

X_test_1 = np.random.multivariate_normal(mean=means[1], cov=covariance_1, size=int(test_size*priors[1]))
# X_test_1 = np.random.multivariate_normal(mean=means[1], cov=covariance_1, size=int(test_size*priors[1]))
# X_test_1 = np.append(X_test_1, (X_test_1[:, 0]**2).reshape(-1, 1), axis=1)
# X_test_1 = np.append(X_test_1, (X_test_1[:, 1]**2).reshape(-1, 1), axis=1)
# X_test_1 = np.append(X_test_1, np.multiply(X_test_1[:, 0], X_test_1[:, 1]).reshape(-1, 1), axis=1)

X_train = np.vstack((X_train_0, X_train_1))
X_test = np.vstack((X_test_0, X_test_1))

y_train = np.vstack((
    np.repeat(0, int(train_size*priors[0])).reshape(-1, 1),
    np.repeat(1, int(train_size*priors[1])).reshape(-1, 1)
))
y_test = np.vstack((
    np.repeat(0, int(test_size*priors[0])).reshape(-1, 1),
    np.repeat(1, int(test_size*priors[1])).reshape(-1, 1)
))

M_0 = X_train_0.mean(axis=0)
M_1 = X_train_1.mean(axis=0)

cov_0 = np.zeros((D, D))
for i in range(X_train_0.shape[0]):
    W = (X_train_0[i] - M_0).reshape(-1, 1)
    cov_0 += np.matmul(W, W.T)

cov_1 = np.zeros((D, D))
for i in range(X_train_1.shape[0]):
    W = (X_train_1[i] - M_1).reshape(-1, 1)
    cov_1 += np.matmul(W, W.T)

S_w = cov_0 + cov_1
S_w_inv = np.linalg.pinv(S_w)
W = np.matmul(S_w_inv, (M_1 - M_0)).reshape(-1, 1)
W = W/np.linalg.norm(W,2)

def predict(x, W, b):
    return np.matmul(W.T, x) + b


# Line Search
best_b, best_acc = 0, 0
lower = np.matmul(W.T, M_0)[0] - 5
upper = np.matmul(W.T, M_1)[0] + 5
for b in np.linspace(lower, upper, 1000):
    acc = 0
    for (i, X_i) in enumerate(X_train_0):
        pred = predict(X_i.reshape(-1, 1), W, b)
        if pred < 0:
            acc += 1

    for (i, X_i) in enumerate(X_train_1):
        pred = predict(X_i.reshape(-1, 1), W, b)
        if pred >= 0:
            acc += 1

    if acc > best_acc:
        best_acc = acc
        best_b = b
    
    # print(f'Bias {b}, Acc: {acc/X_train.shape[0]}, Best: {best_acc/X_train.shape[0]}')

print('Train acc. =', best_acc/X_train.shape[0])

b = best_b
acc = 0
for (i, X_i) in enumerate(X_test_0):
    pred = predict(X_i.reshape(-1, 1), W, b)
    if pred < 0:
        acc += 1

for (i, X_i) in enumerate(X_test_1):
    pred = predict(X_i.reshape(-1, 1), W, b)
    if pred >= 0:
        acc += 1

print(f'Test acc. = {acc/X_test.shape[0]}')

# f1-score
y_pred = []
for X_i in X_train:
    pred = predict(X_i.reshape(-1, 1), W, b)
    if pred < 0:
        y_pred.append(0)
    else:
        y_pred.append(1)
y_pred = np.array(y_pred)
print('F1-Score (train) = ', f1_score(y_train, y_pred))

y_pred = []
for X_i in X_test:
    pred = predict(X_i.reshape(-1, 1), W, b)
    if pred < 0:
        y_pred.append(0)
    else:
        y_pred.append(1)
y_pred = np.array(y_pred)

print('F1-Score (test) = ', f1_score(y_test, y_pred))
confusion_matrix(y_test, y_pred)

Train acc. = 0.9985
Test acc. = 0.995
F1-Score (train) =  0.9984984984984985
F1-Score (test) =  0.9949849548645938


array([[998,   2],
       [  8, 992]])

In [16]:
x = np.linspace(-3, 8, 100)
y = -(W[0][0]*x + b)/(W[1][0])

fig = px.scatter(x=X_train_0[:, 0], y=X_train_0[:, 1])
fig.add_scatter(x=X_train_1[:, 0], y=X_train_1[:, 1], mode='markers', name='class_1')
fig.add_scatter(x=x, y=y, name='decision boundary')